In [1]:
import gzip
import pickle
import pandas as pd
from Bio import SeqIO
from pathlib import Path
from tqdm import tqdm
from gtfparse import read_gtf

pd.options.mode.copy_on_write = True


In [ ]:
# NOT USING AS IT MAY BE MEMORY INEFFICIENT - FILES ARE ~ HALF THE SIZE OF FASTAS

def find_all_NGG_pam_sites(chrom, fasta_path):
    with gzip.open(fasta_path, "rt") as handle:
        fasta_str = str(SeqIO.read(handle, "fasta").seq).upper()

    pam_sites = []
    for i in tqdm(range(len(fasta_str)-2)):
        potential_pam = fasta_str[i:i+3]
        if potential_pam[1:] == "GG":
            strand = "+"
            pam_start_loc = i + 1 # convert to 1 indexing
        elif potential_pam[:-1] == "CC":
            strand = "-"
            pam_start_loc = i + 3 # convert to 1 indexing
        else:
            continue
        # get all variants in the window
        pam_sites.append({"pam_start_loc": pam_start_loc, "strand": strand})

    pam_sites_df = pd.DataFrame(pam_sites) #.sort_values("pam_start_loc")
    pam_sites_df.to_pickle(f"./pam_sites/{chrom}_pam_sites.pkl")
    return

def preprocess_pams():
    # list all genome fastas
    genome_fastas = list(Path("./genome_fastas").rglob("*.fa.gz"))

    # iterate through them
    for fasta_path in genome_fastas:
        chrom = fasta_path.stem.split(".")[0]
        print(chrom)
        find_all_NGG_pam_sites(chrom, f"./genome_fastas/{chrom}.fa.gz")
    # after all are done, run gzip *.pkl in terminal to compress all the pam_sites files



In [2]:

# download hg38 gtf from https://www.ncbi.nlm.nih.gov/datasets/taxonomy/9606/
# read in gtf file
#gtf = pd.read_csv("./hg38.gtf", sep="\t", comment="#", header=None)
gtf_df = read_gtf("../genome_files/hg38.gtf.gz")


# remove all duplicate transcript ids
#gtf["transcript_id"] = gtf[1].str.split(" ", expand=True)[1].str.replace('"', "")
#gtf = gtf.drop_duplicates("transcript_id")
print(gtf_df)



INFO:root:Extracted GTF attributes: ['gene_id', 'transcript_id', 'db_xref', 'description', 'gbkey', 'gene', 'gene_biotype', 'pseudo', 'product', 'transcript_biotype', 'exon_number', 'gene_synonym', 'model_evidence', 'tag', 'protein_id', 'experiment', 'inference', 'note', 'part', 'exception', 'isoform', 'anticodon', 'partial', 'The', 'transl_except', 'non-AUG', 'standard_name', 'deleted', 'source', 'similar', 'substituted', 'transferase', 'codons', '12S', '16S', 'transl_table', 'ATPase']


shape: (4_697_665, 44)
┌──────────────┬────────┬────────────┬───────┬───┬─────┬─────┬──────────────┬────────┐
│ seqname      ┆ source ┆ feature    ┆ start ┆ … ┆ 12S ┆ 16S ┆ transl_table ┆ ATPase │
│ ---          ┆ ---    ┆ ---        ┆ ---   ┆   ┆ --- ┆ --- ┆ ---          ┆ ---    │
│ cat          ┆ str    ┆ cat        ┆ i64   ┆   ┆ str ┆ str ┆ str          ┆ str    │
╞══════════════╪════════╪════════════╪═══════╪═══╪═════╪═════╪══════════════╪════════╡
│ NC_000001.11 ┆        ┆ gene       ┆ 11874 ┆ … ┆     ┆     ┆              ┆        │
│ NC_000001.11 ┆        ┆ transcript ┆ 11874 ┆ … ┆     ┆     ┆              ┆        │
│ NC_000001.11 ┆        ┆ exon       ┆ 11874 ┆ … ┆     ┆     ┆              ┆        │
│ NC_000001.11 ┆        ┆ exon       ┆ 12613 ┆ … ┆     ┆     ┆              ┆        │
│ NC_000001.11 ┆        ┆ exon       ┆ 13221 ┆ … ┆     ┆     ┆              ┆        │
│ …            ┆ …      ┆ …          ┆ …     ┆ … ┆ …   ┆ …   ┆ …            ┆ …      │
│ NC_012920.1  ┆    

In [15]:
# keep only relevant columns for transcripts
gtf_df2 = gtf_df[['gene_id', 'transcript_id', 'feature', 'start', 'end', 'tag', 'strand']]
gtf_df2 = pd.DataFrame(gtf_df2)
gtf_df2.columns = ['gene_id', 'transcript_id', 'feature', 'start', 'end', 'tag', 'strand']

gtf_tx = gtf_df2[gtf_df2['feature'] == 'transcript']
gtf_tx = gtf_tx.drop(columns=['feature'])
gtf_tx = gtf_tx.drop_duplicates("transcript_id")
gtf_tx['MANE'] = gtf_tx.apply(lambda x: -1 if "MANE" in x['tag'] else 0, axis=1)
gtf_tx['transcript_id'] = gtf_tx.apply(lambda x: x['transcript_id'] + " (MANE)" if x['MANE'] == -1 else x['transcript_id'], axis=1)

# keep minimal data in csvs
# sort by transcript id
gtf_tx = gtf_tx.sort_values(['gene_id', 'MANE', "transcript_id"])
# group by gene id and turn transcript ids into a list
gtf_tx_prnt = gtf_tx.groupby("gene_id").agg({"transcript_id": list}).reset_index()
gtf_tx_prnt.to_csv("hg38_transcripts.tsv", sep="\t", index=False)

# create df with transcript id and start stop coords
gtf_coords = gtf_df2

# Filter for transcripts with start and stop codons or exons
transcripts_with_codons = gtf_coords[(gtf_coords['feature'] == 'start_codon') | (gtf_coords['feature'] == 'stop_codon') | (gtf_coords['feature'] == 'exon')]

# Group by transcript_id and aggregate the necessary information
coding_coords = transcripts_with_codons.groupby('transcript_id').apply(lambda x: {
    'transcript_id': x['transcript_id'].iloc[0],
    'strand': x['strand'].iloc[0],
    'length': abs(x[x['feature'] == 'stop_codon']['end'].astype(int).max() - x[x['feature'] == 'start_codon']['start'].astype(int).min()) if 'start_codon' in x['feature'].values and 'stop_codon' in x['feature'].values else abs(x[x['feature'] == 'exon']['end'].astype(int).max() - x[x['feature'] == 'exon']['start'].astype(int).min()),
    'exon_starts': (x[x['feature'] == 'exon']['start'].astype(int) - x[x['feature'] == 'exon']['start'].astype(int).min()).tolist() if len(x[x['feature'] == 'exon']) > 1 else None
}).tolist()

coding_coords_df = pd.DataFrame(coding_coords)
coding_coords_df

#gtf_df4.to_csv("hg38_transcript_coords.tsv", sep="\t", index=False)

KeyboardInterrupt: 

In [ ]:
# create df with transcript id and start stop coords
gtf_coords = gtf_df2
coding_coords = []
for tx in tqdm(gtf_tx['transcript_id'], total=len(gtf_tx)):
    tx_coords = gtf_coords[gtf_coords['transcript_id'] == tx]
    if tx_coords.empty:
        continue
    tx_entry = {"transcript_id": tx, 'strand': tx_coords['strand'].values[0]}

    # get coding sequence length
    if 'start_codon' in tx_coords['feature'].values and 'stop_codon' in tx_coords['feature'].values:
        start = tx_coords[tx_coords['feature'] == 'start_codon']['start'].values[0]
        stop = tx_coords[tx_coords['feature'] == 'stop_codon']['end'].values[0]
        tx_entry["length"] = abs(stop - start)
    elif 'exon' in tx_coords['feature'].values:
        start = tx_coords[tx_coords['feature'] == 'exon']['start'].min()
        stop = tx_coords[tx_coords['feature'] == 'exon']['end'].max()
        tx_entry["length"] = abs(stop - start)
    else:
        print(tx)
        print(tx_coords)
        print("no start stop")
        continue

    # get exon boundaries
    exons = tx_coords[tx_coords['feature'] == 'exon']
    exon_starts = exons['start'] - min(start, stop)
    exon_starts = exon_starts.tolist()
    if len(exon_starts) > 1:
        tx_entry["exon_starts"] = exon_starts   
        
    coding_coords.append(tx_entry)
pd.DataFrame(coding_coords)

In [10]:
app_dir = Path().parent
gene_data = pd.read_csv(app_dir / "genome_files/hg38_transcripts.tsv", sep="\t")
gene_names = list(gene_data["gene_id"])
gene_data['transcript_id'] = gene_data['transcript_id'].apply(eval)
gene_data_dict = gene_data.set_index("gene_id")['transcript_id'].to_dict()

In [12]:
list(gene_data_dict['CHD2'])

['NM_001271.4 (MANE)', 'NM_001042572.3']